In [6]:
import visdom
import torch

vis = visdom.Visdom(env='Test')

x = torch.arange(1, 30, 0.01)
y = torch.sin(x)
vis.line(X=x, Y=y, win='sin', opts={'title': 'y=sin(x)'})

'sin'

#### 代码分析
- vis = visdom.Visdom(env=u'test1')，用于构建一个客户端，客户端除指定env之外，还可以指定host、port等参数
- vis作为一个客户端对象，可以使用常见的画图函数，包括：
    - line 类似Matlab中的plot操作，用于记录某些标量的变化，如损失、准确率等
    - image 可视化图片，可以是输入的图片，也可以是GAN生成的图片，还可以是卷积核的信息
    - text 用于记录日志等文字信息，支持html格式
    - histgram 可视化分布，主要是查看数据、参数的分布
    
Visdom同时支持PyTorch的tensor和Numpy的ndarray两种数据结构，但不支持Python的int、float等类型，因此每次传入时都需先将数据转成ndarray或tensor。上述操作的参数一般不同，但有两个参数是绝大多数操作都具备的：
- win 用于指定pane的名字，如果不指定，visdom将自动分配一个新的pane，名字一样会覆盖
- opts 选项，接收一个字典，常见的option包括`title`、`xlabel`、`ylabel`、`width`等，主要用于设置pane的显示格式

之前提到过，每次操作都会覆盖之前的数值，但往往我们在训练网络的过程中需不断更新数值，如损失值等，这时就需要指定参数`update='append'`来避免覆盖之前的数值。而除了使用update参数以外，还可以使用`vis.updateTrace`方法来更新图，但`updateTrace`不仅能在指定pane上新增一个和已有数据相互独立的Trace，还能像`update='append'`那样在同一条trace上追加数据。

In [7]:
#append 追加数据
for ii in range(0, 10):
    #y = x
    x = torch.Tensor([ii])
    y = x
    vis.line(X=x, Y=y, win='polynomial', update='append' if ii>0 else None)
#updateTrace新增一条线
x = torch.arange(0, 9, 0.1)
y = (x**2)/9
vis.updateTrace(X=x, Y=y, win='polynomial', name='this is a new Trace')

'polynomial'

image的画图功能可分为如下两类：
- image 接收一个二维或三维向量,H×W或3×H×W，前者是黑白图像，后者是彩色图像
- images 接收一个四维向量N×C×H×W,C可以是1或3，分别代表黑白和彩色图像。可实现类似torchvision中make_grid的功能，将多张图片拼接在一起。images也可以接收一个二维或三维的向量，此时它所实现的功能与image一致。

In [9]:
#可视化一个随机的黑白图片
vis.image(torch.randn(64, 64).numpy())
#随机可视化一张彩色图片
vis.image(torch.randn(3, 64, 64).numpy(), win='random2')
#可视化36张随机的彩色图片，每一行6张
vis.images(torch.randn(36, 3, 64, 64).numpy(), nrow=6, win='random3', 
           opts={'title': 'random_imgs'})

'random3'